### Map optimization output back to shapefile
The functions below prepare the data for the process of mapping the results back to their original shapefiles by implementing a "table join" between the outputs .txt and the attribute table of the shapefile

In [1]:
import pandas as pd
import os, sys
#sys.path.append(r'C:\Users\gost_\Desktop\lima\GOSTNets\GOSTNets')
sys.path.append(r'../../../GOSTNets/GOSTNets')
import GOSTnet as gn
import importlib
importlib.reload(gn)
import geopandas as gpd
import rasterio
from rasterio import features
from shapely.wkt import loads
import numpy as np

networkx version: 2.2 
osmnx version: 0.9 
networkx version: 2.2 
osmnx version: 0.9 


In [6]:
#define the paths to the origins, destinations, optimization result
#pth = r'C:\Users\gost_\Desktop\lima'
pth = r'../../../../lima_optimization_output'
walk_speed = 4 
WGS = {'init':'epsg:4326'}
measure_crs = {'init':'epsg:32718'}

The process for mapping the optimization results is as follows:
    1. we need to filter the snapped destination file by the IDs that were generated from the optimization process as the optimal locations
    2. we need to slice the OD matrix by the new destination file, so that only the columns that represent the optimal selected facilities remain
    3. we map the OD matrix back to the origins shapefile as in Step4

In [16]:
#read in the original destination file (from Step3)
destinations_df = pd.read_csv(os.path.join(pth, 'destinations_snapped.csv'))

In [17]:
destinations_df[:5]

,Unnamed: 0,Field1,departamen,provincia,distrito,categoria,Lat,Lon,O_ID,geometry,NN,NN_dist
0,0,337,LIMA,LIMA,VILLA EL SALVADOR,12,-12.248749,-76.930702,337,POINT (-76.93070221000001 -12.24874878),6691,17.668383
1,1,338,LIMA,LIMA,VILLA EL SALVADOR,12,-12.208811,-76.955727,338,POINT (-76.95572661999999 -12.20881081),6048,16.737332
2,2,339,LIMA,LIMA,VILLA EL SALVADOR,14,-12.230375,-76.923637,339,POINT (-76.92363739 -12.23037529),3914,46.775556
3,3,340,LIMA,LIMA,VILLA EL SALVADOR,13,-12.212154,-76.939438,340,POINT (-76.93943787000001 -12.21215439),917,38.959049
4,4,341,LIMA,LIMA,VILLA EL SALVADOR,12,-12.230355,-76.911362,341,POINT (-76.91136169000001 -12.23035526),4919,20.278953


In [28]:
#include the path of the optimization result file *.txt and open the file
#rpath = r'C:\Users\gost_\Desktop\lima\data\OD_distance\lima_OD_distance_output'
Rfile = os.path.join(pth, 'results.txt')
with open (Rfile, 'r') as re:
    results = re.readlines()
results =[(el.strip()) for el in results]
results = [int(i) for i in results]
results [:5]

[2048, 3409, 4154, 6107]

In [29]:
#filter optimal destinations by the result of the optimization process
destinations = destinations_df[destination_df['NN'].isin(result)]
destinations[:100]

,Unnamed: 0,Field1,departamen,provincia,distrito,categoria,Lat,Lon,O_ID,geometry,NN,NN_dist
5,5,342,LIMA,LIMA,VILLA EL SALVADOR,12,-12.224347,-76.922035,342,POINT (-76.92203522 -12.22434711),4154,56.753151
7,7,344,LIMA,LIMA,VILLA EL SALVADOR,12,-12.201385,-76.948044,344,POINT (-76.94804382 -12.2013855),3409,23.288640
13,13,351,LIMA,LIMA,VILLA EL SALVADOR,12,-12.228789,-76.942665,351,POINT (-76.9426651 -12.22878933),2048,19.978944
15,15,353,LIMA,LIMA,VILLA EL SALVADOR,12,-12.213234,-76.944588,353,POINT (-76.94458770999999 -12.21323395),6107,19.211260


In [30]:
#write optimal destination file back to .csv for mapping purposes
destinations.to_csv(os.path.join(pth, 'optimal_destination.csv'))

In [31]:
# read OD matrix
OD = pd.read_csv(os.path.join(pth, r'saved_OD.csv'), sep=',')
OD = OD.rename(columns = {'Unnamed: 0':'NN'})
OD_ = OD.set_index('NN')
OD = OD_.replace([np.inf, -np.inf], np.nan)
OD['Minimum'] = OD.loc[:,:].min(axis=1)

In [43]:
#transform the destination nodes into "string" type to be used as column headers
results_s = [str(x) for x in results]
# filter OD matrix by results
OD = OD[results_s]
OD[:5]

,2048,3409,4154,6107
NN,,,,
6147,1020.721567,1879.396717,819.749517,1584.512721
2052,558.842693,1019.720100,1591.881656,544.361984
3,448.775456,1815.601711,1380.165021,1236.519452
6154,1138.247790,576.396716,1700.636369,822.268208
6162,907.367997,698.388780,1663.935343,592.350060


In [44]:
#adding a column that has the minimum time to reach each origin's nearest facility
OD['Minimum'] = OD.loc[:,:].min(axis=1)
OD[:5]

,2048,3409,4154,6107,Minimum
NN,,,,,
6147,1020.721567,1879.396717,819.749517,1584.512721,819.749517
2052,558.842693,1019.720100,1591.881656,544.361984,544.361984
3,448.775456,1815.601711,1380.165021,1236.519452,448.775456
6154,1138.247790,576.396716,1700.636369,822.268208,576.396716
6162,907.367997,698.388780,1663.935343,592.350060,592.350060


In [46]:
### PREPARE ORIGIN FILE 
# set index to nearest node to match on the travel time from that node to nearest destination (OD['Minimum'])
orig = pd.read_csv(os.path.join(pth, 'origins_snapped.csv'))
#orig = orig.rename({'ID':'OBJECTID'}, axis = 1) # rename ID column to 'OBJECTID'
orig = orig.set_index('NN') # used for matching on the OD matrix
orig[:5]

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN_dist
NN,,,,,,,,,,,
6528,0,1,3.0,0.003205,2.860800e-07,0,1078.0,-12.24839,-76.91749,POINT (-76.91749 -12.24839),21.347245
5270,1,2,3.0,0.008010,2.565700e-06,1,2374.0,-12.24319,-76.92767,POINT (-76.92766999999999 -12.24319),37.506920
1921,2,3,3.0,0.010808,2.141710e-06,2,367.0,-12.24444,-76.91685,POINT (-76.91685 -12.24444),6.260416
3047,3,4,3.0,0.004083,7.910200e-07,3,1230.0,-12.24269,-76.93012,POINT (-76.93012 -12.24269),34.625022
4378,4,5,3.0,0.006058,1.065810e-06,4,169.0,-12.24249,-76.92808,POINT (-76.92808000000001 -12.24249),27.773978


In [47]:
#join works on the index as default 
join = orig.join(OD)
join

,Unnamed: 0,O_ID,NC_CLASS,Shape_Leng,Shape_Area,ORIG_FID,Population,Lat,Lon,geometry,NN_dist,2048,3409,4154,6107,Minimum
NN,,,,,,,,,,,,,,,,
3,633,634,4.0,0.028739,8.435590e-06,633,1458.0,-12.23372,-76.94772,POINT (-76.94771999999999 -12.23372),19.332569,448.775456,1815.601711,1380.165021,1236.519452,448.775456
21,488,489,3.0,0.005872,1.814450e-06,488,2232.0,-12.19822,-76.96106,POINT (-76.96106 -12.19822),63.811570,1618.362899,680.047647,1814.873341,1073.514728,680.047647
32,89,90,3.0,0.005751,1.013350e-06,89,2041.0,-12.23260,-76.91658,POINT (-76.91658000000001 -12.2326),26.145981,1344.498828,1514.702896,455.055695,1511.728358,455.055695
82,549,550,3.0,0.003212,6.469200e-07,549,1508.0,-12.19633,-76.94200,POINT (-76.94199999999999 -12.19633),48.884915,1763.357739,466.052172,950.055216,973.267778,466.052172
84,212,213,3.0,0.003279,6.724400e-07,212,1610.0,-12.22342,-76.94540,POINT (-76.94540000000001 -12.22342),45.891048,316.917488,1138.474966,1271.913024,549.633480,316.917488
99,422,423,3.0,0.004647,1.284970e-06,422,1295.0,-12.20459,-76.95295,POINT (-76.95295 -12.20459),21.629497,1201.410318,280.090546,1428.506362,603.017870,280.090546
106,112,113,3.0,0.009651,3.008850e-06,112,1216.0,-12.23132,-76.93962,POINT (-76.93961999999999 -12.23132),47.702821,215.854603,1501.480910,936.049040,912.770797,215.854603
114,307,308,3.0,0.013462,4.455160e-06,307,824.0,-12.21750,-76.93547,POINT (-76.93547 -12.2175),21.477545,695.977487,995.954493,827.006689,547.879949,547.879949
124,581,582,3.0,0.008233,2.820590e-06,581,440.0,-12.19100,-76.95468,POINT (-76.95468000000001 -12.191),55.011704,1922.036999,651.092484,1499.035247,1097.740446,651.092484


In [49]:
# add on walktime (kph) for time computation
walkspeed = 4
join['walktime'] = join['NN_dist'] / 1000 / walkspeed * 3600
join['total_time'] = (join['walktime'] + join['Minimum']) / 60
###   OR   ###
#compute total distance: NN_dist + dist
join['total_distance'] = (join['NN_dist'] + join['Minimum'])

In [50]:
#read in the origins in shapefile format
fp = r'../../Peru_Lima_Optimization/test_data'
shpfil = gpd.read_file(os.path.join(fp, 'VillaElSalvador_urban.shp'))

In [51]:
# Ensure both shapefile AND origin file have a non-duplicate index to enable matching of results on to the shapefile
shpfil = shpfil.set_index('OBJECTID')
join = join.set_index('O_ID')

In [52]:
# Prep output shapefile, match on visualization column, save
shpfil['total_time'] = join['total_time']
shpfil = shpfil.to_crs({'init':'epsg:4326'})
shpfil.to_file(os.path.join(pth, 'VES_origins_w_time.shp'), driver = 'ESRI Shapefile')